# Solid torus

In this notebook, we will use the `IntervalConstraintProgramming.jl` package to create a solid torus, which is the set $S$ given by the implicit equation

$$S = \{(x,y,z) \in \mathbb{R}^3 : (3 - \sqrt(x^2 + y^2))^2 + z^2 \le 1 \}$$

## Inner and outer approximations of the set

We first calculate inner and outer approximations of the set $S$.

In [19]:
using IntervalConstraintProgramming, ValidatedNumerics

In [20]:
solid_torus = @constraint (3 - sqrt(x^2 + y^2))^2 + z^2 <= 1

Separator:
- variables: x, y, z
- expression: (3 - sqrt(x ^ 2 + y ^ 2)) ^ 2 + z ^ 2 ∈ [-∞, 1]


In [21]:
Y = IntervalBox(-5..5, -5..5, -5..5)
@time paving = pave(solid_torus, Y, 1);

  0.191575 seconds (552.57 k allocations: 20.425 MB, 6.52% gc time)


In [22]:
@time refine!(paving, 0.125)
length(paving.inner)

  8.686054 seconds (34.36 M allocations: 1.247 GB, 7.36% gc time)


14880

In [23]:
paving

Paving:
- tolerance ϵ = 0.125
- inner approx. of length 14880
- boundary approx. of length 15664

`paving.inner` gives an **inner approximation** of $S$, i.e. a union of boxes $X_i$ that are guaranteed to be strictly *inside* $S$, i.e. $X_i \subseteq S$.

`paving.boundary` are boxes that cannot be guaranteed to be inside or outside $S$, but such that the union of `paving.inner` and `paving.boundary` gives an **outer approximation**, i.e. 

$$S \subseteq \bigcup_i X_i \cup \bigcup_b X_b,$$

where the $X_i$ run over the boxes in the inner approximation, and $X_b$ over the boxes in the boundary.

## 3D visualization with `GLVisualize`

We use the `GLVisualize` package to visualize the boxes in 3D. 

We first load the necessary packages:

In [5]:
using GLVisualize, GeometryTypes, GLAbstraction, Colors

We define the positions, box sizes and colours:

In [12]:
inner = paving.inner
positions = Point{3, Float32}[(mid(x)) for x in inner]
scales = Vec3f0[Vec3f0([diam(x) for x in xx]) for xx in inner]
colors = [RGBA(1f0, 0f0, 0f0, 0.1f0) for xx in inner];

And, in only 5 lines of code, we create the 3D visualization in terms of "particles" (in this case, `HyperRectangle`s representing cuboids):

In [14]:
cube_particle = HyperRectangle{3, Float32}(Vec3f0(-0.5, -0.5, -0.5), Vec3f0(1, 1, 1))

window = glscreen()

vis = visualize( (cube_particle, positions), scale=scales, color=colors)

_view(vis)

@async renderloop(window)

Task (runnable) @0x00000001193a4fd0